[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anytko/ecospat/blob/main/docs/examples/persistence_raster.ipynb)

## Steps and pipeline for creating and visualizing a persistence raster for a species given it's modern distribution, northward movement, and population density change.

### Step-by-step

In [ ]:
import ecospat.ecospat as ecospat_full
from ecospat.stand_alone_functions import (
    process_species_historical_range,
    analyze_species_distribution,
    analyze_northward_shift,
    calculate_rate_of_change_first_last,
    merge_category_dataframes,
    prepare_gdf_for_rasterization,
    cat_int_mapping,
    rasterize_multiband_gdf_match,
    rasterize_multiband_gdf_world,
    compute_propagule_pressure_range,
    save_raster_to_downloads_range,
    full_propagule_pressure_pipeline,
)

#### First, we need to classify the historical range edges.

In [ ]:
hist_pipeline = ecospat_full.Map()
hist_range = process_species_historical_range(
    new_map=hist_pipeline, species_name="Populus angustifolia"
)

#### Then we need to classify the modern range edges.

In [ ]:
classified_modern, classified_historic = analyze_species_distribution(
    "Populus angustifolia", record_limit=1000
)

#### Next we will calculate the northward rate of change. We will also clean this dataframe to only include the movement for leading, core, and trailing populations. 

In [ ]:
northward_rate_df = analyze_northward_shift(
    gdf_hist=hist_range,
    gdf_new=classified_modern,
    species_name="Populus angustifolia",
)
northward_rate_df = northward_rate_df[
    northward_rate_df["category"].isin(["leading", "core", "trailing"])
]

northward_rate_df["category"] = northward_rate_df["category"].str.title()

#### After, we will calculate the population density change and clean the dataframe to only include density change for leading, core, and trailing populations. 

In [ ]:
change = calculate_rate_of_change_first_last(
    classified_historic, classified_modern, "Populus angustifolia", custom_end_year=2025
)


change = change[change["collapsed_category"].isin(["leading", "core", "trailing"])]
change = change.rename(
    columns={
        "collapsed_category": "Category",
        "rate_of_change_first_last": "Rate of Change",
        "start_time_period": "Start Years",
        "end_time_period": "End Years",
    }
)


change["Category"] = change["Category"].str.title()

#### We will then merge these dataframes and prepare them for persistence raster creation.

In [ ]:
merged = merge_category_dataframes(northward_rate_df, change)

preped_gdf = prepare_gdf_for_rasterization(classified_modern, merged)

preped_gdf_new = cat_int_mapping(preped_gdf)

preped_gdf_new.head()

#### Once the data is preped, we will rasterize each element into one raster object of 4 different bands. We can either rasterize just the range of the species (rasterize_multiband_gdf_match) or we can extend this raster to the entire world (rasterize_multiband_gdf_world).

In [ ]:
value_columns = [
    "density",
    "northward_rate_km_per_year",
    "Rate of Change",
    "category_int",
]
raster_show, transform, show_bounds = rasterize_multiband_gdf_match(
    preped_gdf_new, value_columns
)

In [ ]:
import matplotlib.pyplot as plt

# Plotting one of these bands (northward movement rate)

plt.imshow(raster_show[1], cmap="viridis", origin="upper")
plt.colorbar(label="Pressure")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

#### After rasterizing the data, we can now construct the persistence raster.

In [ ]:
pressure_show = compute_propagule_pressure_range(raster_show)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(pressure_show, cmap="viridis", origin="upper")
plt.colorbar(label="Pressure")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

#### (Optional) We can also save this raster as a .tif

In [ ]:
# raster_download = save_raster_to_downloads_range(pressure_show, show_bounds, "Populus angustifolia")

#### To display this saved .tif raster we can use the .add_raster method. There is already an example raster download within the package to add here.

In [ ]:
persistence_map = ecospat_full.Map()
persistence_map.add_basemap("GBIF.Classic")
persistence_map.add_raster(
    "Populus_angustifolia_persistence_raster.tif",
    colormap="viridis",
    legend=True,
    name="Persistence Raster",
)
# persistence_map.add_layer_control()
persistence_map

### Pipeline to generate persistence raster 

#### For the simplified pipeline, we will still need to fetch the gbif data, calculate the northward movement, and the population density change.

In [ ]:
full_show, full_save, show_bounds, save_bounds, gdf_transform, world_transform = (
    full_propagule_pressure_pipeline(classified_modern, northward_rate_df, change)
)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(full_show, cmap="viridis", origin="upper")
plt.colorbar(label="Pressure")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()